In [10]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [11]:
import numpy as np
import pandas as pd
import datetime as dt

In [12]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [13]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///../Resources/hawaii.sqlite")

In [14]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(autoload_with=engine)
Base.classes.keys()

['measurement', 'station']

In [15]:
# Save references to each table
measurement = Base.classes.measurement

station = Base.classes.station

In [16]:
# Create our session (link) from Python to the DB

session = Session(bind=engine)

In [17]:
# Find the most recent date in the data set.

most_recent_date = session.query(func.max(measurement.date)).scalar()

print("Most recent date: ",most_recent_date)

Most recent date:  2017-08-23


In [18]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.
max_date_query = "SELECT MAX(date) FROM hawaii.sqlite"
max_date_query = pd.read_sql_query(max_date_query, engine) ['MAX(date))'][0]
one_year_ago = pd.to_datetime(max_date) - pd.DateOffset(years=1)

# Perform a query to retrieve the data and precipitation scores
query = f"SELECT date, precipitation FROM hawaii.sqlite WHERE date >= '{one_year_ago}'"
df = pd.read_sql_query(query, engine)

# Save the query results as a Pandas DataFrame. Explicitly set the column names
df.columns = ['Date', 'Precipitation']

# Sort the dataframe by date
df = df.sort_values('Date')

# Use Pandas Plotting with Matplotlib to plot the data
plt.figure(figsize=(10,6))
plt.plot(df['date'], df['Precipitation'])
plt.xlabel('Date')
plt.ylabel('Inches')
plt.title('Precipitation')
plt.xticks(rotation=45)
plt,tight_layout()
plt.show()

OperationalError: (sqlite3.OperationalError) no such table: hawaii.sqlite
[SQL: SELECT MAX(date) FROM hawaii.sqlite]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
# Use Pandas to calculate the summary statistics for the precipitation data


In [ ]:
# Design a query to calculate the total number of stations in the dataset


In [ ]:
# Design a query to find the most active stations (i.e. which stations have the most rows?)
# List the stations and their counts in descending order.


In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


In [ ]:
# Close Session
session.close()